In [1]:
pip install pandas==1.3.1

In [2]:
pip install pyLDAvis

In [3]:
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
import nltk
from nltk.corpus import stopwords
import gensim
from gensim import corpora

In [4]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = dataset.data
print('샘플의 수 :',len(documents))

샘플의 수 : 11314


In [6]:
documents[1]

"\n\n\n\n\n\n\nYeah, do you expect people to read the FAQ, etc. and actually accept hard\natheism?  No, you need a little leap of faith, Jimmy.  Your logic runs out\nof steam!\n\n\n\n\n\n\n\nJim,\n\nSorry I can't pity you, Jim.  And I'm sorry that you have these feelings of\ndenial about the faith you need to get by.  Oh well, just pretend that it will\nall end happily ever after anyway.  Maybe if you start a new newsgroup,\nalt.atheist.hard, you won't be bummin' so much?\n\n\n\n\n\n\nBye-Bye, Big Jim.  Don't forget your Flintstone's Chewables!  :) \n--\nBake Timmons, III"

In [7]:
print(dataset.target_names)

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [8]:
news_df = pd.DataFrame({'document':documents})
# 특수 문자 제거
news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z]", " ")
# 길이가 3이하인 단어는 제거 (길이가 짧은 단어 제거)
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
# 전체 단어에 대한 소문자 변환
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: x.lower())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until


In [9]:
news_df['clean_doc'][1]

'yeah expect people read actually accept hard atheism need little leap faith jimmy your logic runs steam sorry pity sorry that have these feelings denial about faith need well just pretend that will happily ever after anyway maybe start newsgroup atheist hard bummin much forget your flintstone chewables bake timmons'

In [10]:
# NLTK로부터 불용어를 받아온다.
stop_words = stopwords.words('english')
tokenized_doc = news_df['clean_doc'].apply(lambda x: x.split()) # 토큰화
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])

In [11]:
tokenized_doc[:5]

0    [well, sure, story, seem, biased, disagree, st...
1    [yeah, expect, people, read, actually, accept,...
2    [although, realize, principle, strongest, poin...
3    [notwithstanding, legitimate, fuss, proposal, ...
4    [well, change, scoring, playoff, pool, unfortu...
Name: clean_doc, dtype: object

In [12]:
dictionary = corpora.Dictionary(tokenized_doc)
corpus = [dictionary.doc2bow(text) for text in tokenized_doc]

# 수행된 결과에서 1번 인덱스 뉴스 출력
print(corpus[1])

[(52, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 2), (67, 1), (68, 1), (69, 1), (70, 1), (71, 2), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 2), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 2), (86, 1), (87, 1), (88, 1), (89, 1)]


In [13]:
print(dictionary[66])

faith


In [14]:
len(dictionary)

64281

In [15]:
NUM_TOPICS = 20
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)

topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.015*"image" + 0.011*"file" + 0.009*"color" + 0.008*"jpeg"')
(1, '0.012*"health" + 0.010*"medical" + 0.006*"disease" + 0.006*"among"')
(2, '0.010*"windows" + 0.008*"thanks" + 0.008*"system" + 0.008*"like"')
(3, '0.010*"pain" + 0.009*"ground" + 0.008*"wire" + 0.008*"food"')
(4, '0.014*"would" + 0.013*"people" + 0.009*"know" + 0.009*"think"')
(5, '0.023*"jesus" + 0.011*"church" + 0.011*"bible" + 0.010*"christian"')
(6, '0.014*"team" + 0.012*"play" + 0.011*"hockey" + 0.010*"period"')
(7, '0.016*"armenian" + 0.013*"said" + 0.013*"armenians" + 0.009*"people"')
(8, '0.005*"washington" + 0.005*"university" + 0.005*"national" + 0.005*"california"')
(9, '0.011*"would" + 0.011*"people" + 0.007*"government" + 0.005*"right"')
(10, '0.011*"nrhj" + 0.007*"wwiz" + 0.006*"bxom" + 0.006*"gizw"')
(11, '0.010*"wiring" + 0.008*"nist" + 0.007*"object" + 0.006*"ncsl"')
(12, '0.009*"would" + 0.008*"year" + 0.007*"game" + 0.007*"good"')
(13, '0.012*"guns" + 0.009*"crime" + 0.006*"koresh" + 0.006*"contro

In [16]:
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(ldamodel, corpus, dictionary)
pyLDAvis.display(vis)

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


In [17]:
for i, topic_list in enumerate(ldamodel[corpus]):
  if i==5:
    break
  print(i,'번째 문서의 topic 비율은',topic_list)

0 번째 문서의 topic 비율은 [(4, 0.25231537), (9, 0.5775969), (15, 0.08552773), (17, 0.071656734)]
1 번째 문서의 topic 비율은 [(1, 0.025977176), (4, 0.89367014), (11, 0.025905473), (12, 0.034934994)]
2 번째 문서의 topic 비율은 [(4, 0.36350933), (9, 0.5720972), (15, 0.03810212), (18, 0.013388125)]
3 번째 문서의 topic 비율은 [(2, 0.026832117), (9, 0.61615765), (12, 0.3443237)]
4 번째 문서의 topic 비율은 [(6, 0.16325445), (12, 0.55060077), (16, 0.08470782), (17, 0.17180736)]


In [18]:
def make_topictable_per_doc(ldamodel, corpus):
    topic_table = pd.DataFrame()

    # 몇 번째 문서인지를 의미하는 문서 번호와 해당 문서의 토픽 비중을 한 줄씩 꺼내온다.
    for i, topic_list in enumerate(ldamodel[corpus]):
        doc = topic_list[0] if ldamodel.per_word_topics else topic_list            
        doc = sorted(doc, key=lambda x: (x[1]), reverse=True)
        # 각 문서에 대해서 비중이 높은 토픽순으로 토픽을 정렬한다.
        # EX) 정렬 전 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (10번 토픽, 5%), (12번 토픽, 21.5%), 
        # Ex) 정렬 후 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (12번 토픽, 21.5%), (10번 토픽, 5%)
        # 48 > 25 > 21 > 5 순으로 정렬이 된 것.

        # 모든 문서에 대해서 각각 아래를 수행
        for j, (topic_num, prop_topic) in enumerate(doc): #  몇 번 토픽인지와 비중을 나눠서 저장한다.
            if j == 0:  # 정렬을 한 상태이므로 가장 앞에 있는 것이 가장 비중이 높은 토픽
                topic_table = topic_table.append(pd.Series([int(topic_num), round(prop_topic,4), topic_list]), ignore_index=True)
                # 가장 비중이 높은 토픽과, 가장 비중이 높은 토픽의 비중과, 전체 토픽의 비중을 저장한다.
            else:
                break
    return(topic_table)

In [19]:
topictable = make_topictable_per_doc(ldamodel, corpus)
topictable = topictable.reset_index() # 문서 번호을 의미하는 열(column)로 사용하기 위해서 인덱스 열을 하나 더 만든다.
topictable.columns = ['문서 번호', '가장 비중이 높은 토픽', '가장 높은 토픽의 비중', '각 토픽의 비중']
topictable[:10]

,문서 번호,가장 비중이 높은 토픽,가장 높은 토픽의 비중,각 토픽의 비중
0,0,9.0,0.5776,"[(4, 0.2523168), (9, 0.57759565), (15, 0.08552..."
1,1,4.0,0.8937,"[(1, 0.025977172), (4, 0.893699), (11, 0.02590..."
2,2,9.0,0.5721,"[(4, 0.363506), (9, 0.5721176), (15, 0.0381004..."
3,3,9.0,0.6162,"[(2, 0.026808413), (9, 0.6161733), (12, 0.3443..."
4,4,12.0,0.5507,"[(6, 0.16318975), (12, 0.5507354), (16, 0.0846..."
5,5,4.0,0.3889,"[(4, 0.38891304), (5, 0.2271887), (6, 0.052206..."
6,6,7.0,0.6820,"[(1, 0.03519127), (2, 0.18253222), (4, 0.07333..."
7,7,4.0,0.4419,"[(4, 0.44189575), (8, 0.16993888), (9, 0.37467..."
8,8,9.0,0.4583,"[(1, 0.18368359), (7, 0.22147726), (9, 0.45830..."
9,9,18.0,0.4506,"[(2, 0.14051956), (4, 0.052282523), (5, 0.0202..."
